### ***User Manual***
- Go to [API - Google AI Studio](https://aistudio.google.com/apikey) to get the API key.
- Install the necessary libraries 
    - `pip install -q -U google-genai`
    - `pip install python-dotenv tqdm pillow`
- Create a `.env` file in the same dir as the notebook with the content `GEMINI_API_KEY = "YOUR_API_KEY"`
- Hàm chính cần quan tâm là `load_generation` vs cái `prompt` bên trong `generate` là ổn nhé.
    - Chạy cái hàm `load_generation` là được.
    - Có thể đọc phần docstring ở hàm `load_generation` để tham khảo.

### ***API Limitation***
Hiện tại chúng ta đang dùng của Gemini free nên cơ bản mình sẽ bị những thứ sau:
- 15 RPM (Request per minute)
- 1 mil TPM (Total per minute)
- 1500 RPD (Request per day)

In [ ]:
import json
import os
import re
from time import sleep

from google import genai
from PIL import Image
from dotenv import load_dotenv
from tqdm.auto import tqdm

# Load environment variables from a .env file. This is often used to store sensitive information like API keys.
load_dotenv()
# Define a tuple containing common image file extensions. This might be used later to identify image files.
IMAGE_EXT = (".jpg", ".jpeg", ".png", ".gif", ".webp", ".tiff")

# Configure the Google Generative AI library with the API key obtained from the environment variables.
CLIENT = genai.Client(api_key = os.getenv("GEMINI_API_KEY"))
# Marking using the "gemini-2.0-flash" model.
# This model can be used for various generative tasks, including text and image generation.
MODEL = "gemini-2.0-flash"

In [ ]:
def generate(img_path, num = 5):
    prompt = f"""
    Bạn là một chuyên gia phân tích các số liệu thống kê từ infographic tin tức.  
    Dựa vào hình ảnh được cung cấp, hãy đề xuất các cặp câu hỏi - câu trả lời liên quan đến việc đếm các đối tượng thoả mãn một tiêu chí nào đó  
    Hãy thể hiện sự sáng tạo của bạn trong việc đặt câu hỏi và trả lời.

    Các quy tắc cần tuân theo:
    - Bao gồm {int(num * 0.6)} câu hỏi yêu cầu đếm các đối tượng dựa trên một tính chất liên quan đến các yếu tố dạng text (số liệu, thông tin dạng văn bản, ...). Nếu câu hỏi liên quan đến số liệu, câu hỏi được yêu cầu phải thêm các thông tin như lớn hơn hoặc nhỏ hơn 1 giá trị nào đó hoặc là trong 1 khoảng giá trị nào đó. Nếu câu hỏi liên quan đến việc đếm số tên gọi, cầu hỏi được yêu cầu phải thêm thông tin như tên gọi đó bắt đầu bằng 1 từ hoặc 1 cụm từ cụ thể nào đó.
    - Bao gồm {int(num * 0.4)} câu hỏi yêu cầu đếm các đối tượng dựa trên một tính chất liên quan đến các yếu tố dạng non-text (các yếu tố phi văn bản như hình dạng biểu đồ, vị trí trên bản đồ, các vật thể (con người, cây cối, ...), ...). 
    - Trước khi đặt những câu hỏi, hãy quan sát infographic trước và tìm ra những nội dụng chính mà tấm infographic đó muốn truyền tải, từ đó đặt những câu hỏi liên quan đến nội dung chính đó. 
    - Không được phép đặt những câu hỏi thiếu sự chi tiết, quá tổng quát và khó định danh chính xác thông tin trên infographic để trả lời (ví dụ: Không được đặt là "Có bao nhiêu hình người xuất hiện trong infographic?" mà nên đặt là "Có bao nhiêu hình người xuất hiện hình bên trái ở phần "..." của Infographic?") 
    - Câu hỏi và câu trả lời phải ở dạng câu hoàn chỉnh, đồng thời, câu trả lời phải được viết rõ ràng, đề cập đầy đủ các ý được hỏi từ câu hỏi. 
    - Chỉ sử dụng thông tin có trong infographic, không bổ sung kiến thức nền tảng bên ngoài. 
    - Không tạo câu hỏi có dạng có/không hoặc dạng lựa chọn.
 
    - Không đặt câu hỏi yêu cầu phân tích sâu hoặc suy luận ngoài dữ liệu infographic.
    - Không được đặt những câu hỏi mà không đủ dữ kiện hay số liệu để đưa ra câu trả lời tương ứng.
    - Xử lý các biểu đồ, đồ thị, bản đồ và các yếu tố phi văn bản một cách hợp lý.
    - Câu hỏi và câu trả lời không được quá 30 từ.
    - Thêm thông tin sau vào mỗi câu trả lời:
      + Explanation: Ở phần này, bạn hãy đưa ra lý do cụ thể (không quá 100 từ) rằng tại sao bạn lại đưa ra câu trả lời như thế, vì là các câu hỏi liên quan đến việc đếm nên lúc này, bạn hãy chỉ rõ tên của các đối tượng được đếm trong lời giải thích của bạn. Yêu cầu: Lời giải thích phải được viết thành đoạn văn, không được gạch đầu dòng. 
    - Không đưa ra những câu hỏi có thể trả lời câu hỏi đó mà không cần nhìn vào infographic (ví dụ: "Trong 4 quốc gia Việt Nam, Mông Cổ, Myanmar, Malaysia, quốc gia nào không có tên bắt đầu bằng chữ M", ở câu này bạn không cần phải nhìn vào infographic vẫn có thể trả lời chính xác, điều này là không được phép!)
    
    Hãy trả lời theo cấu trúc sau:
    ---
    Q: (câu hỏi)  
    A: (câu trả lời)  
    Explanation: (Lời giải thích cho câu trả lời của bạn) 
    Type: (Text nếu câu hỏi yêu cầu đếm các yếu tố dạng text hoặc Non-text nếu câu hỏi yêu cầu đếm các yếu tố dạng non-text)
    ---
    """

    img = Image.open(img_path)
    response = CLIENT.models.generate_content(
        model=MODEL,
        contents=[prompt, img],
    )
    return response.text

def clean(text: str):
    """
    Cleans a text string by removing extra spaces and replacing double quotes with single quotes.

    Args:
        text (str): The text string to clean.

    Returns:
        str: The cleaned text string.
    """
    patterns = {
        r"\s+": " ",  # replace multiple spaces with a single space
        r"\"": "'",  # replace double quotes with single quotes to avoid JSON parsing error
    }
    text = text.strip()
    for pattern, repl in patterns.items():
        text = re.sub(pattern, repl, text)
    return text


def extract(response: str, start_ques = "Q: ", start_ans = "A: ", start_exp = "Explanation: ", start_type = "Type: "):
    """
    Extracts questions, answers, explanations, and types from the generated text response.

    Args:
        response (str): The generated text containing the question-answer pairs.
        start_ques (str, optional): The starting string for questions. Defaults to "Q: ".
        start_ans (str, optional): The starting string for answers. Defaults to "A: ".
        start_exp (str, optional): The starting string for explanations. Defaults to "Explanation: ".
        start_type (str, optional): The starting string for types. Defaults to "Type: ".

    Returns:
        list: A list of dictionaries, where each dictionary contains a question, answer, explanation, and type.
    """
    questions = []
    answers = []
    explanations = []
    types = []

    for line in response.split("\n"):
        if line.startswith(start_ques):
            questions.append(clean(line[len(start_ques) : ]))
        elif line.startswith(start_ans):
            answers.append(clean(line[len(start_ans) : ]))
        elif line.startswith(start_exp):
            explanations.append(clean(line[len(start_exp) : ]))
        elif line.startswith(start_type):
            types.append(clean(line[len(start_type) : ]))

    return [
        {
            "Question": q,
            "Answer": a,
            "Explanation": e,
            "Type": t
        }
        for q, a, e, t in zip(questions, answers, explanations, types)
    ]

In [ ]:
def load_generation(folder: str, num: int = 5, prefix: str = "qa_", sleep_time: int = 1):
    """
    Load images from a specified folder and perform a generation process
    (presumably generating questions and answers related to the images).

    Process:
    - Reads existing generated data from a JSON file (creates a new one if it doesn't exist).
    - Identifies images in the folder that haven't been processed yet.
    - Generates questions and answers for each unprocessed image.

    Parameters:
    - folder (str): The path to the folder containing the image files.
    - num (int): The number of question-answer pairs to generate for each image (total 2*num generated).
    - prefix (str): The prefix to be used for the name of the JSON file where the generated data is saved.
    - sleep_time (int): The number of seconds to wait between processing each image.

    Returns:
    - None. The generated data is saved to a JSON file.
    """
    def save_json(data, qa_file):
        """
        Saves the generated data to a JSON file.

        Parameters:
        - data (dict): The dictionary containing the generated data.
        - qa_file (str): The path to the JSON file to save the data to.

        Returns:
        - None
        """
        with open(qa_file, "w", encoding="utf-8") as f:
            json.dump(data, f, ensure_ascii=False, indent=4)
        print(f"[+] Saved to {qa_file}")

    data = {}
    qa_file = prefix + folder + ".json"
    # Check if the JSON file exists and is not empty
    if os.path.isfile(qa_file) and os.path.getsize(qa_file) > 0:
        # Load existing data from the JSON file
        with open(qa_file, "r", encoding="utf-8") as f:
            data = json.load(f)
    else:
        # Create a new empty JSON file if it doesn't exist
        with open(qa_file, "w", encoding="utf-8") as f:
            pass

    # Get a list of image files in the specified folder that have not been processed yet
    images = [f for f in os.listdir(folder) if f.endswith(IMAGE_EXT) and f not in data]
    # If there are no new images to process
    if not images:
        print("[+] All done!")
        return

    try:
        # Iterate through the unprocessed images with a progress bar
        with tqdm(total=len(images), unit=" in4graphic", dynamic_ncols=True) as pbar:
            for img in images:
                pbar.set_description(f"Processing {img}")
                img_path = os.path.join(folder, img)
                # Retry loop for handling potential errors, especially "Too Many Requests"
                while True:
                    try:
                        # Call the 'generate' function (assumed to be defined elsewhere)
                        # to generate data for the current image.
                        # Then, call the 'extract' function (assumed to be defined elsewhere)
                        # to process the generated data.
                        data[img] = extract(generate(img_path, num=num))
                        pbar.update(1)
                        sleep(sleep_time)
                        break  # Exit the retry loop on successful processing
                    except Exception as e:
                        # Handle "Too Many Requests" error by waiting and retrying
                        if "429" in str(e):
                            print("[-] Received 429 Too Many Requests. Retrying after 10 seconds...")
                            sleep(10)
                        else:
                            raise  # Re-raise other exceptions that are not rate-limiting errors
        # Save the updated data to the JSON file after processing all images
        save_json(data, qa_file)

    # Handle potential exceptions during the process, including keyboard interrupts
    except (Exception, KeyboardInterrupt) as e:
        print(f"[-] Error: {e if not isinstance(e, KeyboardInterrupt) else 'Interrupted'}")
        # Save the current state of the data to the JSON file in case of an error or interruption
        save_json(data, qa_file)

# Example usage of the function
load_generation("Part3", num = 5, sleep_time = 1)